In [10]:
import os
from openai import OpenAI
import gradio as gr
from dotenv import load_dotenv
load_dotenv()

SECRET_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=SECRET_KEY)

system_prompt = """As a movie lover, I watch a lot of films. 
If you have any queries, feel free to ask, and I can also translate the information into a language other than English if needed.
You will ensure to talk only about movies/ films."""
# non-streaming
def generate_response(message, history):
    formatted_history = []
    for user, assistant in history:
        formatted_history.append({"role": "system", "content": system_prompt })
        formatted_history.append({"role": "user", "content": user })
        formatted_history.append({"role": "assistant", "content":assistant})

    formatted_history.append({"role": "user", "content": message})
  
    response = client.chat.completions.create(
    model='gpt-4o',
    messages= formatted_history,
    temperature=0.2, 
    max_tokens = 100
    )

    return response.choices[0].message.content


# streaming example - it does not work
def generate_streaming_response(message, history):
    history_openai_format = []
    for human, assistant in history:
        history_openai_format.append({"role": "system", "content": system_prompt})
        history_openai_format.append({"role": "user", "content": user })
        history_openai_format.append({"role": "assistant", "content":assistant})
    history_openai_format.append({"role": "user", "content": message})
  
    response = client.chat.completions.create(model='gpt-3.5-turbo',
    messages= history_openai_format,
    temperature=1.0,
    stream=True)

    partial_message = ""
    for chunk in response:
        if chunk.choices[0].delta.content is not None:
              partial_message = partial_message + chunk.choices[0].delta.content
              yield partial_message

gr.ChatInterface(fn = generate_response,
    chatbot=gr.Chatbot(height=300, placeholder="<strong>Name a film.</strong><br>Language of your choice.</br>"),
    textbox=gr.Textbox(placeholder="Which movie do you learn about?", container=False, scale=7),
    title="Movie Lovers Pub Crawl",
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
    theme="monochrome",
    analytics_enabled = False
    ).launch()


Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.
